In [69]:
# %% [markdown]
# # Подготовка данных для проекта "СберАвтоподписка"
# 
# ## 1. Импорт библиотек

# %%
import pandas as pd
import numpy as np
from pathlib import Path
import os
import logging

# Настройка логирования
logging.basicConfig(filename='data_processing.log', 
                    level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

# %% [markdown]
# ## 2. Вспомогательные функции

# %%
def generate_summary(df: pd.DataFrame, name: str) -> None:
    """Генерация сводки по данным"""
    print(f"\n{'='*40} {name} SUMMARY {'='*40}")
    print(f"Общее количество записей: {df.shape[0]:,}")
    print(f"Количество признаков: {df.shape[1]}")
    print("\nТипы данных:")
    print(df.dtypes.value_counts().to_frame('count'))
    
    print("\nПропущенные значения:")
    missing = df.isna().sum()
    print(missing[missing > 0].to_frame('count'))

# %% [markdown]
# ## 3. Основные функции обработки

# %%
def load_data() -> tuple:
    """Загрузка и конвертация данных"""
    try:
        current_dir = Path(os.getcwd()).absolute()
        root_dir = current_dir.parent.parent if 'notebooks' in current_dir.parts else current_dir
        DATA_PATH = root_dir / 'data' / 'raw_data'
        PROCESSED_PATH = root_dir / 'data' / 'processed_data'
        
        DATA_PATH.mkdir(parents=True, exist_ok=True)
        PROCESSED_PATH.mkdir(parents=True, exist_ok=True)

        # Конвертация CSV в PKL
        if not (DATA_PATH / 'ga_sessions.pkl').exists():
            pd.read_csv(DATA_PATH / 'ga_sessions.csv').to_pickle(DATA_PATH / 'ga_sessions.pkl')
        
        if not (DATA_PATH / 'ga_hits.pkl').exists():
            pd.read_csv(DATA_PATH / 'ga_hits.csv').to_pickle(DATA_PATH / 'ga_hits.pkl')

        return (
            pd.read_pickle(DATA_PATH / 'ga_sessions.pkl'),
            pd.read_pickle(DATA_PATH / 'ga_hits.pkl'),
            PROCESSED_PATH
        )
    
    except Exception as e:
        logging.error(f"Ошибка загрузки: {str(e)}", exc_info=True)
        raise

def process_time(df: pd.DataFrame, date_col: str, time_col: str) -> pd.DataFrame:
    """Универсальная обработка временных меток"""
    def fix_time_format(t):
        try:
            if pd.isna(t):
                return np.nan
            t = str(t).strip()
            if t in ['24:00:00', '2400', '24:00']:
                return '00:00:00'
            if len(t.split(':')) == 2:
                return f'{t}:00'
            return t
        except:
            return np.nan

    # Преобразование даты и времени
    df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
    
    if time_col in df.columns:
        df[time_col] = (
            df[time_col]
            .apply(fix_time_format)
            .apply(lambda x: pd.to_datetime(x, format='%H:%M:%S', errors='coerce').time()
                  if pd.notnull(x) else pd.NaT)
        )
    
    # Создание полной метки времени
    if date_col and time_col in df.columns:
        df['full_datetime'] = df.apply(
            lambda row: pd.NaT if pd.isnull(row[date_col]) or pd.isnull(row[time_col])
            else pd.Timestamp.combine(row[date_col], row[time_col]), 
            axis=1
        )
    
    # Валидация и очистка
    if 'full_datetime' in df.columns:
        mask = (df['full_datetime'].dt.year < 2020) | (df['full_datetime'].dt.year > 2023)
        df.loc[mask, 'full_datetime'] = pd.NaT
        df = df.dropna(subset=['full_datetime']).copy()
    
    return df

def process_resolution(df: pd.DataFrame) -> pd.DataFrame:
    """Обработка разрешения экрана"""
    def _parse(res_str):
        try:
            res_str = str(res_str).lower().strip()
            if not res_str or res_str in ['(not set)', 'nan', '0']:
                return 0, 0
            if 'x' not in res_str:
                return 0, 0
            w, h = res_str.split('x', 1)
            return int(float(w)), int(float(h))
        except:
            return 0, 0
    
    df[['screen_w', 'screen_h']] = df['device_screen_resolution'].apply(
        lambda x: pd.Series(_parse(x)))
    return df.drop('device_screen_resolution', axis=1)

def handle_missing(df: pd.DataFrame) -> pd.DataFrame:
    """Обработка пропущенных значений"""
    # Устройства
    device_map = {
        'device_os': 'Unknown',
        'device_brand': 'Unknown',
        'device_model': 'Generic'
    }
    for col, default in device_map.items():
        df[col] = df[col].fillna(default).replace({'(not set)': default})
    
    # UTM-метки
    utm_rules = {
        'utm_source': {'(none)': 'organic'},
        'utm_medium': {'(none)': 'organic'},
        'utm_campaign': {'(not set)': 'general'},
        'utm_adcontent': {'(not set)': 'none'},
        'utm_keyword': {'(not set)': 'none'}
    }
    for col, mapping in utm_rules.items():
        df[col] = df[col].replace(mapping).fillna('other')
    
    return df

def add_conversion(sessions: pd.DataFrame, hits: pd.DataFrame) -> pd.DataFrame:
    """Добавление целевой переменной"""
    target_actions = ['sub_button_click', 'sub_page_view', 'sub_view_cars_click']
    hits['conversion'] = hits['event_action'].isin(target_actions).astype(int)
    return sessions.merge(
        hits.groupby('session_id')['conversion'].max(),
        left_on='session_id',
        right_index=True,
        how='left'
    ).fillna({'conversion': 0})

# %% [markdown]
# ## 4. Главная функция

# %%
def main():
    """Основной пайплайн обработки данных"""
    # Загрузка данных
    sessions, hits, processed_path = load_data()
    
    # Обработка временных меток
    sessions = process_time(sessions, 'visit_date', 'visit_time')
    hits = process_time(hits, 'hit_date', 'hit_time')
    
    generate_summary(sessions, "ИСХОДНЫЕ СЕССИИ")
    generate_summary(hits, "ИСХОДНЫЕ СОБЫТИЯ")
    
    # Основная обработка
    sessions = (sessions
                .pipe(process_resolution)
                .pipe(handle_missing)
                .pipe(add_conversion, hits))
    
    # Финализация
    generate_summary(sessions, "ОБРАБОТАННЫЕ СЕССИИ")
    
    # Сохранение
    sessions.to_pickle(processed_path / 'processed_sessions.pkl')
    hits.to_pickle(processed_path / 'processed_hits.pkl')
    print("\n✅ Данные успешно обработаны и сохранены!")

# %% [markdown]
# ## 5. Запуск обработки

# %%
if __name__ == "__main__":
    main()

ValueError: NaTType does not support time